In [1]:
# neuroevolution_mlp_ga.ipynb
import numpy as np
import random
from sklearn.datasets import load_digits
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

random.seed(7); np.random.seed(7)
POP_SIZE = 20
GENERATIONS = 15
TOURNAMENT_K = 3
CROSSOVER_RATE = 0.9
MUTATION_RATE = 0.3
ELITISM = True

X, y = load_digits(return_X_y=True)

ACTIVATIONS = ["relu", "tanh", "logistic"]
UNITS_RANGE = (16, 128)   # neuronas por capa
LAYERS_RANGE = (1, 3)     # 1 a 3 capas ocultas

def init_individual():
    n_layers = np.random.randint(LAYERS_RANGE[0], LAYERS_RANGE[1]+1)
    units = [np.random.randint(*UNITS_RANGE) for _ in range(n_layers)]
    return {
        "layers": n_layers,
        "units": units,       # lista, ej. [64, 32]
        "activation": random.choice(ACTIVATIONS),
    }

def to_hidden_tuple(ind):
    return tuple(ind["units"])

def fitness(ind):
    clf = Pipeline([
        ("scaler", StandardScaler()),
        ("mlp", MLPClassifier(
            hidden_layer_sizes=to_hidden_tuple(ind),
            activation=ind["activation"],
            max_iter=120,
            early_stopping=True,
            n_iter_no_change=5,
            random_state=7))
    ])
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=7)
    return cross_val_score(clf, X, y, cv=cv, scoring="accuracy").mean()

def tournament_select(pop, fits):
    idxs = np.random.choice(len(pop), size=TOURNAMENT_K, replace=False)
    best = max(idxs, key=lambda i: fits[i])
    # clonar (profundo) para listas
    return {"layers": pop[best]["layers"],
            "units": pop[best]["units"][:],
            "activation": pop[best]["activation"]}

def uniform_crossover(a, b):
    if np.random.rand() >= CROSSOVER_RATE:
        return a, b
    c1, c2 = {}, {}
    # capas
    if np.random.rand() < 0.5:
        c1_layers, c2_layers = a["layers"], b["layers"]
    else:
        c1_layers, c2_layers = b["layers"], a["layers"]
    # unidades (alinear longitudes con padding simple)
    maxL = max(len(a["units"]), len(b["units"]), c1_layers, c2_layers)
    au = (a["units"] + [a["units"][-1]]*(maxL-len(a["units"]))) if a["units"] else [32]*maxL
    bu = (b["units"] + [b["units"][-1]]*(maxL-len(b["units"]))) if b["units"] else [32]*maxL
    c1_units = []
    c2_units = []
    for i in range(maxL):
        if np.random.rand() < 0.5:
            c1_units.append(au[i]); c2_units.append(bu[i])
        else:
            c1_units.append(bu[i]); c2_units.append(au[i])
    c1_units = c1_units[:c1_layers]
    c2_units = c2_units[:c2_layers]

    # activación
    c1_act = a["activation"] if np.random.rand() < 0.5 else b["activation"]
    c2_act = a["activation"] if np.random.rand() >= 0.5 else b["activation"]

    c1 = {"layers": c1_layers, "units": c1_units, "activation": c1_act}
    c2 = {"layers": c2_layers, "units": c2_units, "activation": c2_act}
    return c1, c2

def mutate(ind):
    out = {"layers": ind["layers"], "units": ind["units"][:], "activation": ind["activation"]}
    # cambiar nº de capas
    if np.random.rand() < MUTATION_RATE:
        out["layers"] = int(np.clip(out["layers"] + np.random.choice([-1, 1]), *LAYERS_RANGE))
        # ajustar lista de unidades
        while len(out["units"]) < out["layers"]:
            out["units"].append(np.random.randint(*UNITS_RANGE))
        while len(out["units"]) > out["layers"]:
            out["units"].pop()
    # mutar unidades
    for i in range(out["layers"]):
        if np.random.rand() < MUTATION_RATE:
            out["units"][i] = int(np.clip(out["units"][i] + np.random.randint(-16, 17), *UNITS_RANGE))
    # mutar activación
    if np.random.rand() < MUTATION_RATE:
        out["activation"] = random.choice(ACTIVATIONS)
    return out

# GA
population = [init_individual() for _ in range(POP_SIZE)]
fitnesses = [fitness(ind) for ind in population]
for gen in range(GENERATIONS):
    new_pop = []
    if ELITISM:
        elite = population[int(np.argmax(fitnesses))]
        new_pop.append({"layers": elite["layers"], "units": elite["units"][:], "activation": elite["activation"]})
    while len(new_pop) < POP_SIZE:
        p1 = tournament_select(population, fitnesses)
        p2 = tournament_select(population, fitnesses)
        c1, c2 = uniform_crossover(p1, p2)
        c1, c2 = mutate(c1), mutate(c2)
        new_pop.extend([c1, c2])
    population = new_pop[:POP_SIZE]
    fitnesses = [fitness(ind) for ind in population]
    print(f"Gen {gen+1:02d} | best={np.max(fitnesses):.4f} | avg={np.mean(fitnesses):.4f}")

best = population[int(np.argmax(fitnesses))]
print("Best architecture:", best)
print("Best CV accuracy:", np.max(fitnesses))


ModuleNotFoundError: No module named 'sklearn'